In [1]:
path = r"C:\Users\sirish\Downloads\ravdess-emotional-speech-audio"
lst = []

import os
import librosa
import numpy as np


for subdir, dirs, files in os.walk(path):
  for file in files:
      try:
        #Load librosa array, obtain mfcss, store the file and the mcss information in a new array
        X, sample_rate = librosa.load(os.path.join(subdir,file), res_type='kaiser_fast')
        mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T,axis=0) 
        # The instruction below converts the labels (from 1 to 8) to a series from 0 to 7
        # This is because our predictor needs to start from 0 otherwise it will try to predict also 0.
        file = int(file[7:8]) - 1 
        arr = mfccs, file
        lst.append(arr)
      # If the file is not valid, skip it
      except ValueError:
        continue

In [2]:
X, y = zip(*lst)

In [3]:
import numpy as np
X = np.asarray(X)
y = np.asarray(y)


X.shape, y.shape

((1440, 40), (1440,))

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20 , random_state=60)              

In [5]:
x_traincnn = np.expand_dims(X_train, axis=2)
x_testcnn = np.expand_dims(X_test, axis=2)

In [6]:

x_traincnn.shape, x_testcnn.shape

((1152, 40, 1), (288, 40, 1))

In [26]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.utils import to_categorical
from keras.layers import Input, Flatten, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint

model = Sequential()

model.add(Conv1D(128, 8,padding='same',
                 input_shape=(40,1)))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=(5)))
model.add(Conv1D(256, 8,padding='same',))
model.add(Activation('relu'))
model.add(MaxPooling1D(pool_size=(5)))
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(8))
model.add(Activation('softmax'))
opt = keras.optimizers.rmsprop(lr=0.00003, rho=0.9, decay=0.0)

In [27]:
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [28]:
cnnhistory=model.fit(x_traincnn, y_train, batch_size=16, epochs=700, validation_data=(x_testcnn, y_test))




Train on 1152 samples, validate on 288 samples
Epoch 1/700
1152/1152 [==============================] - 3s 2ms/step - loss: 5.3277 - accuracy: 0.1198 - val_loss: 2.0891 - val_accuracy: 0.1250
Epoch 2/700
1152/1152 [==============================] - 2s 2ms/step - loss: 3.0374 - accuracy: 0.1562 - val_loss: 1.9808 - val_accuracy: 0.2049
Epoch 3/700
1152/1152 [==============================] - 3s 2ms/step - loss: 2.4496 - accuracy: 0.1736 - val_loss: 2.0801 - val_accuracy: 0.1875
Epoch 4/700
1152/1152 [==============================] - 3s 2ms/step - loss: 2.2319 - accuracy: 0.1866 - val_loss: 1.9194 - val_accuracy: 0.2500
Epoch 5/700
1152/1152 [==============================] - 2s 2ms/step - loss: 2.1148 - accuracy: 0.1988 - val_loss: 1.9242 - val_accuracy: 0.2917
Epoch 6/700
1152/1152 [==============================] - 2s 2ms/step - loss: 2.0547 - accuracy: 0.2196 - val_loss: 1.8447 - val_accuracy: 0.3333
Epoch 7/700
1152/1152 [==============================] - 2s 2ms/step - loss: 2.0221